# COGS 108 - Data Checkpoint

# Names

- Jamie Wei
- Alexis Garduno
- James Daza
- Aleksander Archipov

<a id='research_question'></a>
# Research Question

- Does the number of times a player is traded predict that player's performance (e.g. average total points scored/game, average minutes played/game, etc) in an NBA game? Also, does the turn-over rate (e.g. number of players traded within a team/season) of a NBA team affect the likelihood that the team will reach the NBA finals (evidenced by the last 20 years of NBA games)?

# Dataset(s)

- Dataset Name: nba_api (replace with the dataset we generated from nba_api) 
- Link to the dataset: https://github.com/swar/nba_api
- Number of observations: N/A
    - API Client that allows access to various NBA's stats API endpoints

- Dataset Name: NBA Stats (replace with boxscore and team performance datasets)
- Link to the dataset: https://www.nba.com/stats/
- Number of observations: 12
    - Player stats from 10 random teams ranging from 2016-2021


# Setup

In [2]:
import pandas as pd
import numpy as np
import time

from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats

### Gathering Teams

In [3]:
#Import the roster of teams from the NBA API
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

We select 10 nba teams at random 

In [4]:
#Obtain a full list of all abbreviations - will need abbreviations to identify team statistics
nba_teams_df=pd.DataFrame(nba_teams)
team_id=nba_teams_df['id'] #this is the unique team id
team_id_random=np.random.choice(team_id,10,replace=False) #identify ten team ids
nba_teams_rdf=nba_teams_df[nba_teams_df['id'].isin(team_id_random)] #df of 10 randomly selected teams
nba_teams_rdf

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
11,1610612748,Miami Heat,MIA,Heat,Miami,Florida,1988
13,1610612750,Minnesota Timberwolves,MIN,Timberwolves,Minnesota,Minnesota,1989
20,1610612757,Portland Trail Blazers,POR,Trail Blazers,Portland,Oregon,1970
21,1610612758,Sacramento Kings,SAC,Kings,Sacramento,California,1948
25,1610612762,Utah Jazz,UTA,Jazz,Utah,Utah,1974
26,1610612763,Memphis Grizzlies,MEM,Grizzlies,Memphis,Tennessee,1995
28,1610612765,Detroit Pistons,DET,Pistons,Detroit,Michigan,1948


### Requesting Games
Now we request the games for all of the ten teams we have selected

In [5]:
#Pull all games for all ten teams

#Documentation for this endpoint: 
#https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/leaguegamefinder.md
from nba_api.stats.endpoints import leaguegamefinder

# Query for games from the League Game Finder
gamefinder=pd.DataFrame()
for i in team_id_random:
    time.sleep(1) #delay to prevent being blocked from the API
    df = leaguegamefinder.LeagueGameFinder(team_id_nullable=[i]).get_data_frames()[0] #parameter of team ids given
    gamefinder = pd.concat([df,gamefinder])

In [6]:
#Game Statistics

#One row corresponds to one game and one team.
#There will be two rows per game, since there are two teams that played each other.
#Will need to exclude duplicate rows (XXXX will remove duplicate rows)
print(list(set(gamefinder.TEAM_ID))) #confirmed that identified 10 different teams
print(gamefinder.shape) #31,386 games

##Game Finder Dataset: This dataset will be used as the outcome when we look at the association between 
##the exposure and outcome relationship. 
gamefinder

[1610612737, 1610612739, 1610612741, 1610612748, 1610612750, 1610612757, 1610612758, 1610612762, 1610612763, 1610612765]
(32941, 28)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1610612739,CLE,Cleveland Cavaliers,0022100867,2022-02-15,CLE @ ATL,L,240,116,...,0.833,14.0,25.0,39,30,8.0,3,13,23,-8.0
1,22021,1610612739,CLE,Cleveland Cavaliers,0022100849,2022-02-12,CLE @ PHI,L,240,93,...,0.867,5.0,29.0,34,21,3.0,3,15,16,-10.0
2,22021,1610612739,CLE,Cleveland Cavaliers,0022100839,2022-02-11,CLE @ IND,W,240,120,...,0.833,6.0,26.0,32,20,12.0,7,13,16,7.0
3,22021,1610612739,CLE,Cleveland Cavaliers,0022100825,2022-02-09,CLE vs. SAS,W,242,105,...,0.750,10.0,41.0,51,18,8.0,2,13,12,13.0
4,22021,1610612739,CLE,Cleveland Cavaliers,0022100805,2022-02-06,CLE vs. IND,W,242,98,...,0.679,16.0,34.0,50,25,9.0,5,10,22,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3468,21983,1610612737,ATL,Atlanta Hawks,0028300058,1983-11-06,ATL @ MIL,L,240,84,...,0.667,11.0,26.0,37,18,4.0,5,14,24,NaN
3469,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,...,0.692,19.0,27.0,46,31,14.0,13,18,27,NaN
3470,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,...,0.633,12.0,29.0,41,20,7.0,10,16,34,NaN
3471,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,...,0.633,27.0,21.0,48,28,14.0,7,23,35,NaN


Now we request all the player stats from seasons ranging from 2016-2021

In [7]:
#get game ids for the last five years

#for now, let's focus on the last five seasons for ease
from nba_api.stats.endpoints import playergamelogs

#generate a parameter dataframe to define timeframe
#Is this timeframe correct? What is the timeframe that the season normally runs from?
season_parameter_df=pd.DataFrame({'Season':['2016-17','2017-18','2018-19','2019-20','2020-21'], 
                    'Date_From':['9/01/2016','9/01/2017','9/01/2018','9/01/2019','9/01/2020'],
                    'Date_To':['8/31/2017','8/31/2018','8/31/2019','8/31/2020','8/31/2021']})

#will obtain no game ids, without the season_nullable and date_nullable items selected
logsdf=pd.DataFrame()
for i in list(range(1, 5)):
    time.sleep(1)
    season=season_parameter_df.iloc[i][0]
    date_from=season_parameter_df.iloc[i][1]
    date_to=season_parameter_df.iloc[i][2]
    
    logs = pd.DataFrame(playergamelogs.PlayerGameLogs(
        season_nullable = season,
        date_from_nullable = date_from,                                                     
        date_to_nullable = date_to
    ).player_game_logs.get_data_frame())
    logsdf = pd.concat([logs,logsdf])

In [8]:
#unique set of game ids
game_ids = list(set(logsdf['GAME_ID'])) #not sure if we need game_ids
player_ids = pd.DataFrame(list(set(logsdf['PLAYER_ID'])))
print(pd.DataFrame(game_ids).shape) #4599 game ids
print(player_ids.shape) #875 unique players
print(logsdf.shape) #97,655 rows where each row is for each player in each game. 

(4599, 1)
(875, 1)
(97655, 66)


<b>4599 Games <br>
875 Unique Players <br>
97,655 stats of players in each game<b>

All the columns of every player that played in the seasons we selected from the random teams chosen

In [9]:
#preview, likely not use these columns
logsdf.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2',
       'TD3', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK',
       'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK', 'VIDEO_AVAILABLE_FLAG'],
      dtype='object')

In [10]:
#import boxscore, player statistics within each game
#https://en.wikipedia.org/wiki/Box_score_(baseball)#:~:text=A%20box%20score%20is%20a,the%20box%20score%20in%201858.
#from nba_api.stats.endpoints import BoxScoreAdvancedV2

# Takes a LONG time to run
#boxscfinder=pd.DataFrame()
#for i in game_ids:
#        time.sleep(1) #delay to prevent being blocked from the API
#        boxscore = BoxScoreAdvancedV2(game_id=i).player_stats.get_data_frame()
#        boxscfinder = pd.concat([boxscore,boxscfinder])

In [11]:
####NEXT STEPS:

####These tasks need to be completed and we can split them up.
#Task (Insert your name)

#TASK 1 (NAME): Get a final dataset for the Boxscore (verify that this runs)
#TASK 2 (NAME): Only keep the boxscore entries that match the teams identified
#in the team_id_random ARRAY; generate a file

#TASK 3 (Jamie): Run this notebook and restrict the dataset
#named "boxscfinder" to the parameters we need based on what you 
#identified. Note, Jamie, when you restrict the parameter, it would
#be helpful in the code to name the variables. 

#TASK 4 (NAME): Restrict the "gamefinder" dataframe to the "team performance" metrics

#TASK 5 (Alexis): Look at the final boxscore dataset and define the traded variable

#TASK 6 (Alek/other): Estimate the data distributions of the 
#the parameters that we are examining. 
#Depends on Task 3 and 4 being complete. 
#Generate descriptive statistics for the parameters that we list.


# Data Cleaning

(Alek and others) Describe. In class, data cleaning examples were to handle missing values, recode variables (generate new variables), and plot distribution.<br>

### Cleaning Player Logs

In [222]:
# Copy Logs DataFrame and remove TEAM ID's that were not selected
logsDf = logsdf.copy()


# Restrict to the player stats we want
logsDf = logsDf[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'MIN', 'FG_PCT', 'FT_PCT', 'PTS', 'AST', 'REB', 'STL', 'BLK', 'PLUS_MINUS']]
logsDf


,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,MIN,FG_PCT,FT_PCT,PTS,AST,REB,STL,BLK,PLUS_MINUS
0,2020-21,1626245,Cristiano Felicio,1610612741,Chicago Bulls,17.950000,0.400,0.500,5,0,8,1,0,5
1,2020-21,1630163,LaMelo Ball,1610612766,Charlotte Hornets,33.475000,0.333,0.750,19,5,7,2,0,-2
2,2020-21,1628419,Cameron Oliver,1610612745,Houston Rockets,28.750000,0.556,0.000,10,1,12,0,3,-23
3,2020-21,201145,Jeff Green,1610612751,Brooklyn Nets,22.400000,0.667,1.000,13,0,2,1,1,-12
4,2020-21,1629667,Jalen McDaniels,1610612766,Charlotte Hornets,27.775000,0.571,1.000,13,2,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26102,2017-18,203499,Shane Larkin,1610612738,Boston Celtics,4.816667,0.000,0.000,0,0,1,0,0,2
26103,2017-18,201583,Ryan Anderson,1610612745,Houston Rockets,33.283333,0.417,0.000,13,1,8,1,1,8
26104,2017-18,2594,Kyle Korver,1610612739,Cleveland Cavaliers,7.316667,0.000,0.000,0,0,0,0,0,-1
26105,2017-18,202691,Klay Thompson,1610612744,Golden State Warriors,38.150000,0.429,0.000,16,3,6,2,2,0


### Cleaning boxstats

In [13]:
# NEED TO RUN BOXSTATS REQUEST BLOCK FIRST

# Restrict boxscfinder to teams the random teams we selected
# boxscfinder = boxscfinder[boxscfinder['TEAM_ID'].isin(team_id_random)]

# Restrict to the player stats we want
# boxscfinder = boxscfinder[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'MIN', 'FG_PCT', 'FT_PCT', 'PTS', 'AST', 'REB', 'STL', 'BLK', 'PLUS_MINUS']
# boxscfinder

### Saving Data <br>
Now we save the data we gathered we requested from the NBA API into csv files

In [223]:
#Saving the Data to a csv file
logsDf.to_csv("logPlayerStats.csv")

# NEED TO RUN BOXSTATS REQUEST BLOCK FIRST
# boxscfinder.to_csv("boxscfinder.csv")

### Manipulating/Plotting CSV File Data

In [15]:
import seaborn as sns

In [226]:
#Need to plot the distributions data using either seaborn or matplotlib

dataDf = pd.read_csv("logPlayerStats.csv")
#dataDf['TIMES_TRADED'] = np.nan
pid_lst = list(player_ids[0])
pid_lst[0]





playerStats = pd.DataFrame()
for pid in pid_lst:
    time.sleep(1)
    career = playercareerstats.PlayerCareerStats(player_id=pid)
    temp = career.get_data_frames()[0]
    temp = temp[temp['SEASON_ID'].isin(season)]
    playerStats = playerStats.append(temp)
    

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

...

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  playerStats = playerStats.append(temp)
<ipython-input-226-8326a2494edf>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [227]:
season = ['2016-17','2017-18','2018-19','2019-20','2020-21']
for s in season:
    playerStats["Traded " + s] = 0
    
for pid in pid_lst:
    for s in season:
        tempDf = dataDf.loc[(dataDf['PLAYER_ID'] == pid) & (dataDf['SEASON_YEAR'] == s)]
        tempDf = tempDf.reset_index(drop=True)
        tempDf = tempDf.drop_duplicates(subset=['TEAM_ID'])
        traded = len(tempDf.index)-1
        
        if traded >= 0:
            playerStats.loc[(playerStats['PLAYER_ID'] == pid), ["Traded " + s]] = traded
        else:
            playerStats.loc[(playerStats['PLAYER_ID'] == pid), ["Traded " + s]] = 0

In [228]:
playerStats = playerStats.reset_index(drop=True)
playerStats

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,STL,BLK,TOV,PF,PTS,Traded 2016-17,Traded 2017-18,Traded 2018-19,Traded 2019-20,Traded 2020-21
0,1630208,2020-21,00,1610612766,CHA,23.0,18,0,63.0,4,...,0,0,3,7,15,0,0,0,0,0
1,202714,2016-17,00,1610612762,UTA,27.0,55,9,1205.0,170,...,42,3,90,100,430,0,0,1,0,0
2,202714,2017-18,00,1610612753,ORL,28.0,69,3,1365.0,178,...,54,8,89,91,473,0,0,1,0,0
3,202714,2018-19,00,1610612763,MEM,29.0,53,3,1204.0,161,...,45,4,62,81,421,0,0,1,0,0
4,202714,2018-19,00,1610612766,CHA,29.0,4,0,42.0,2,...,2,0,4,3,9,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,1630203,2020-21,00,1610612766,CHA,24.0,7,0,27.0,8,...,1,0,1,0,18,0,0,0,0,0
3214,1630204,2020-21,00,1610612750,MIN,21.0,2,0,4.0,0,...,0,0,1,0,0,0,0,0,0,0
3215,1630205,2020-21,00,1610612739,CLE,23.0,40,0,499.0,62,...,17,13,23,53,165,0,0,0,0,0
3216,1630206,2020-21,00,1610612746,LAC,20.0,4,1,84.0,14,...,4,0,7,5,35,0,0,0,0,0


In [229]:
playerStats.to_csv("PlayerStats.csv")

In [230]:
playerStats

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,STL,BLK,TOV,PF,PTS,Traded 2016-17,Traded 2017-18,Traded 2018-19,Traded 2019-20,Traded 2020-21
0,1630208,2020-21,00,1610612766,CHA,23.0,18,0,63.0,4,...,0,0,3,7,15,0,0,0,0,0
1,202714,2016-17,00,1610612762,UTA,27.0,55,9,1205.0,170,...,42,3,90,100,430,0,0,1,0,0
2,202714,2017-18,00,1610612753,ORL,28.0,69,3,1365.0,178,...,54,8,89,91,473,0,0,1,0,0
3,202714,2018-19,00,1610612763,MEM,29.0,53,3,1204.0,161,...,45,4,62,81,421,0,0,1,0,0
4,202714,2018-19,00,1610612766,CHA,29.0,4,0,42.0,2,...,2,0,4,3,9,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,1630203,2020-21,00,1610612766,CHA,24.0,7,0,27.0,8,...,1,0,1,0,18,0,0,0,0,0
3214,1630204,2020-21,00,1610612750,MIN,21.0,2,0,4.0,0,...,0,0,1,0,0,0,0,0,0,0
3215,1630205,2020-21,00,1610612739,CLE,23.0,40,0,499.0,62,...,17,13,23,53,165,0,0,0,0,0
3216,1630206,2020-21,00,1610612746,LAC,20.0,4,1,84.0,14,...,4,0,7,5,35,0,0,0,0,0


In [231]:
playerStats[playerStats['PLAYER_ID'] ==1626144 ]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,STL,BLK,TOV,PF,PTS,Traded 2016-17,Traded 2017-18,Traded 2018-19,Traded 2019-20,Traded 2020-21
60,1626144,2016-17,00,1610612743,DEN,21.0,55,41,1406.0,208,...,41,13,123,95,603,0,1,0,0,0
61,1626144,2017-18,00,1610612743,DEN,22.0,42,0,752.0,129,...,21,4,72,47,359,0,1,0,0,0
62,1626144,2017-18,00,1610612752,NYK,22.0,22,14,493.0,75,...,20,7,41,33,194,0,1,0,0,0
63,1626144,2017-18,00,0,TOT,22.0,64,14,1244.0,204,...,41,11,113,80,553,0,1,0,0,0
64,1626144,2018-19,00,1610612752,NYK,23.0,59,42,1607.0,330,...,43,19,140,103,873,0,1,0,0,0
65,1626144,2019-20,00,1610612762,UTA,24.0,54,2,850.0,151,...,24,10,76,72,395,0,1,0,0,0


In [207]:
season_stats = pd.DataFrame()
for pid in pid_lst:
    time.sleep(1)
    career = playercareerstats.PlayerCareerStats(player_id=pid)
    temp = career.career_totals_regular_season.get_data_frame()
    season_stats = season_stats.append(temp)  

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

...

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_stats = season_stats.append(temp)
<ipython-input-207-ff937b8e9632>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

In [208]:
season_stats

,PLAYER_ID,LEAGUE_ID,Team_ID,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630208,00,0,59,5,406.0,59,90,0.655555,0,...,0.711538,34,57,91,15,9,19,24,56,155
0,202714,00,0,456,56,8393.0,1165,2752,0.423328,302,...,0.731462,118,776,894,1421,300,24,560,568,2997
0,1630210,00,0,59,1,339.0,67,179,0.374301,41,...,0.869565,1,26,27,24,11,1,24,34,195
0,1630211,00,0,10,0,88.0,3,13,0.230769,1,...,0.800000,1,13,14,4,0,2,3,10,11
0,1630214,00,0,98,12,1594.0,236,452,0.522123,29,...,0.643564,120,252,372,115,85,47,71,165,566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1630203,00,0,7,0,27.0,8,12,0.666666,2,...,0.000000,0,1,1,3,1,0,1,0,18
0,1630204,00,0,2,0,4.0,0,0,0.000000,0,...,0.000000,0,0,0,0,0,0,1,0,0
0,1630205,00,0,81,8,1085.0,146,313,0.466453,17,...,0.681318,53,134,187,50,37,22,45,112,371
0,1630206,00,0,22,1,206.0,32,82,0.390243,8,...,0.800000,4,26,30,9,8,3,14,14,84


In [210]:
season = ['2016-17','2017-18','2018-19','2019-20','2020-21']
for s in season:
    season_stats["Traded " + s] = 0
    
for pid in pid_lst:
    for s in season:
        tempDf = dataDf.loc[(dataDf['PLAYER_ID'] == pid) & (dataDf['SEASON_YEAR'] == s)]
        tempDf = tempDf.reset_index(drop=True)
        tempDf = tempDf.drop_duplicates(subset=['TEAM_ID'])
        traded = len(tempDf.index)-1
        
        if traded >= 0:
            season_stats.loc[(season_stats['PLAYER_ID'] == pid), ["Traded " + s]] = traded

        else:
            season_stats.loc[(playerStats['PLAYER_ID'] == pid), ["Traded " + s]] = 0

In [211]:
career_stats

,PLAYER_ID,LEAGUE_ID,Team_ID,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,...,STL,BLK,TOV,PF,PTS,Traded 2016-17,Traded 2017-18,Traded 2018-19,Traded 2019-20,Traded 2020-21
0,1630208,00,0,59,5,406.0,59,90,0.655555,0,...,9,19,24,56,155,0,0,0,0,0
0,202714,00,0,456,56,8393.0,1165,2752,0.423328,302,...,300,24,560,568,2997,0,0,1,0,0
0,1630210,00,0,59,1,339.0,67,179,0.374301,41,...,11,1,24,34,195,0,0,0,0,0
0,1630211,00,0,10,0,88.0,3,13,0.230769,1,...,0,2,3,10,11,0,0,0,0,0
0,1630214,00,0,98,12,1594.0,236,452,0.522123,29,...,85,47,71,165,566,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1630203,00,0,7,0,27.0,8,12,0.666666,2,...,1,0,1,0,18,0,0,0,0,0
0,1630204,00,0,2,0,4.0,0,0,0.000000,0,...,0,0,1,0,0,0,0,0,0,0
0,1630205,00,0,81,8,1085.0,146,313,0.466453,17,...,37,22,45,112,371,0,0,0,0,0
0,1630206,00,0,22,1,206.0,32,82,0.390243,8,...,8,3,14,14,84,0,0,0,0,0


In [216]:
carrer_stats[season_stats['Traded 2017-18'] > 0]

,PLAYER_ID,LEAGUE_ID,Team_ID,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,...,STL,BLK,TOV,PF,PTS,Traded 2016-17,Traded 2017-18,Traded 2018-19,Traded 2019-20,Traded 2020-21
0,1626143,00,0,247,116,4826.0,1092,2016,0.541666,6,...,73,195,347,515,2566,0,1,0,0,0
0,1626144,00,0,302,165,7185.0,1222,3051,0.400524,276,...,215,88,670,492,3299,0,1,0,0,0
0,1626195,00,0,287,44,4356.0,820,1528,0.536649,33,...,118,112,287,470,2084,0,1,0,0,0
0,1626204,00,0,387,133,9346.0,1296,2454,0.528117,162,...,479,192,387,867,3144,0,1,0,0,0
0,202732,00,0,177,24,1907.0,129,321,0.401869,43,...,108,31,86,201,346,0,1,0,0,0
0,202328,00,0,642,417,17626.0,3363,6543,0.513984,1,...,678,370,1195,1427,8381,0,2,2,0,0
0,202738,00,0,533,362,15344.0,3115,7170,0.434449,1019,...,465,58,1232,1124,9574,0,1,0,0,0
0,1626246,00,0,280,25,2611.0,635,1104,0.575181,10,...,54,91,187,325,1671,0,1,1,0,0
0,202337,00,0,381,107,5332.0,650,1593,0.408035,394,...,89,58,165,520,1818,0,1,0,0,0
0,2207,00,0,1277,1091,44233.0,7823,17720,0.441478,1978,...,1071,260,2451,2252,20407,0,1,0,0,0


In [232]:
career_stats.to_csv("careerStats.csv")

NameError: name 'career_stats' is not defined